In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import xgboost as xgb
from sklearn.impute import SimpleImputer
from glob import glob
import os
from tqdm import tqdm
from hyperopt import fmin, tpe, hp, Trials
from hyperopt.pyll.base import scope
from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid
from keras.models import Sequential
from keras.layers import Input, LSTM, GRU, Dense
from keras.optimizers import Adam
from prophet import Prophet
import gc

from sklearn.preprocessing import OneHotEncoder

### GreyWolf Optimization for Hyperparameter Tuning
Source: https://github.com/Valdecy/pyMetaheuristic/blob/main/pyMetaheuristic/algorithm/gwo.py

In [ ]:
# Function: Initialize Variables
def initial_variables(size, min_values, max_values, target_function, start_init = None):
    dim = len(min_values)
    if (start_init is not None):
        start_init = np.atleast_2d(start_init)
        n_rows     = size - start_init.shape[0]
        if (n_rows > 0):
            rows       = np.random.uniform(min_values, max_values, (n_rows, dim))
            start_init = np.vstack((start_init[:, :dim], rows))
        else:
            start_init = start_init[:size, :dim]
        fitness_values = target_function(start_init) if hasattr(target_function, 'vectorized') else np.apply_along_axis(target_function, 1, start_init)
        population     = np.hstack((start_init, fitness_values[:, np.newaxis] if not hasattr(target_function, 'vectorized') else fitness_values))
    else:
        population     = np.random.uniform(min_values, max_values, (size, dim))
        fitness_values = target_function(population) if hasattr(target_function, 'vectorized') else np.apply_along_axis(target_function, 1, population)
        population     = np.hstack((population, fitness_values[:, np.newaxis] if not hasattr(target_function, 'vectorized') else fitness_values))
    return population

############################################################################

# Function: Initialize Alpha
def alpha_position(min_values, max_values, target_function):
    alpha       = np.zeros((1, len(min_values) + 1))
    alpha[0,-1] = target_function(np.clip(alpha[0,0:alpha.shape[1]-1], min_values, max_values))
    return alpha[0,:]


# Function: Initialize Beta
def beta_position(min_values, max_values, target_function):
    beta       = np.zeros((1, len(min_values) + 1))
    beta[0,-1] = target_function(np.clip(beta[0,0:beta.shape[1]-1], min_values, max_values))
    return beta[0,:]


# Function: Initialize Delta
def delta_position(min_values, max_values, target_function):
    delta       =  np.zeros((1, len(min_values) + 1))
    delta[0,-1] = target_function(np.clip(delta[0,0:delta.shape[1]-1], min_values, max_values))
    return delta[0,:]


# Function: Updtade Pack by Fitness
def update_pack(position, alpha, beta, delta):
    idx   = np.argsort(position[:, -1])
    alpha = position[idx[0], :]
    beta  = position[idx[1], :] if position.shape[0] > 1 else alpha
    delta = position[idx[2], :] if position.shape[0] > 2 else beta
    return alpha, beta, delta


# Function: Update Position
def update_position(position, alpha, beta, delta, a_linear_component, min_values, max_values, target_function):
    dim                     = len(min_values)
    alpha_position          = np.copy(position)
    beta_position           = np.copy(position)
    delta_position          = np.copy(position)
    updated_position        = np.copy(position)
    r1                      = np.random.rand(position.shape[0], dim)
    r2                      = np.random.rand(position.shape[0], dim)
    a                       = 2 * a_linear_component * r1 - a_linear_component
    c                       = 2 * r2
    distance_alpha          = np.abs(c * alpha[:dim] - position[:, :dim])
    distance_beta           = np.abs(c * beta [:dim] - position[:, :dim])
    distance_delta          = np.abs(c * delta[:dim] - position[:, :dim])
    x1                      = alpha[:dim] - a * distance_alpha
    x2                      = beta [:dim] - a * distance_beta
    x3                      = delta[:dim] - a * distance_delta
    alpha_position[:,:-1]   = np.clip(x1, min_values, max_values)
    beta_position [:,:-1]   = np.clip(x2, min_values, max_values)
    delta_position[:,:-1]   = np.clip(x3, min_values, max_values)
    alpha_position[:, -1]   = np.apply_along_axis(target_function, 1, alpha_position[:, :-1])
    beta_position [:, -1]   = np.apply_along_axis(target_function, 1, beta_position [:, :-1])
    delta_position[:, -1]   = np.apply_along_axis(target_function, 1, delta_position[:, :-1])
    updated_position[:,:-1] = np.clip((alpha_position[:, :-1] + beta_position[:, :-1] + delta_position[:, :-1]) / 3, min_values, max_values)
    updated_position[:, -1] = np.apply_along_axis(target_function, 1, updated_position[:, :-1])
    updated_position        = np.vstack([position, updated_position, alpha_position, beta_position, delta_position])
    updated_position        = updated_position[updated_position[:, -1].argsort()]
    updated_position        = updated_position[:position.shape[0], :]
    return updated_position


def grey_wolf_optimizer(pack_size, min_values, max_values, iterations, target_function, verbose=True):
    alpha = alpha_position(min_values, max_values, target_function)
    beta = beta_position(min_values, max_values, target_function)
    delta = delta_position(min_values, max_values, target_function)
    position = initial_variables(pack_size, min_values, max_values, target_function)

    for count in range(iterations):
        if verbose:
            print(f"Iteration {count}/{iterations}: Best Fitness = {alpha[-1]}")

        a_linear_component = 2 - count * (2 / iterations)
        alpha, beta, delta = update_pack(position, alpha, beta, delta)
        position = update_position(position, alpha, beta, delta, a_linear_component, min_values, max_values, target_function)
        alpha = position[position[:, -1].argsort()][0]

    return map_params(alpha[:-1]), alpha[-1]

### Load Files

In [ ]:
subject_files = glob("/content/drive/MyDrive/Upwork clients/ACCOUNT-CREATOR/DiaTrend Data/DiaTrend Data/Sensor Data/*.xlsx")
print(f"Number of subject files found: {len(subject_files)}")

Number of subject files found: 54


##### Configuration:
##### **Note**: Set `Subjects` and `CGM_Inputs` in Conf class below as per your computation power

In [ ]:
class Conf :
    # dataloading values
    Subjects = 20 # define how many subjects you want to include in processing
    CGM_Inputs = 20000 # define how many CGM inputs (rows) to consider for training

    # when you set iterator as 5 it would take input as a 6 (always plus one than input)
    iterator = 3 # it would be recommended to set less than 10
    iterator_deeplearning = 2 # deep learnig take time so set it to less than 5

    # set agents value (number of wolves)
    number_wolves = 10


### Data Preprocessing

In [ ]:
def create_lag_features(df, column, lags):
    for lag in lags:
        df[f'{column}_lag_{lag}'] = df[column].shift(lag)
    return df

# Placeholder for all data
all_data = []

# Iterate over each subject file
for subject_file in tqdm(sorted(subject_files)[:min(Conf.Subjects, len(subject_files))]):
    try:
        print(f"Processing {subject_file}...")

        # Load CGM and Bolus data for each subject
        cgm_data = pd.read_excel(subject_file, sheet_name='CGM', parse_dates=['date'])
        bolus_data = pd.read_excel(subject_file, sheet_name='Bolus', parse_dates=['date'])

        # Check if data was loaded correctly
        if cgm_data.empty or bolus_data.empty:
            print(f"No data found in {subject_file}")
            continue

        cgm_data['date'] = pd.to_datetime(cgm_data['date'], format='ISO8601')
        bolus_data['date'] = pd.to_datetime(bolus_data['date'], format='ISO8601')

        # Drop columns with high NaN values
        bolus_data.drop(columns=['recommended.carb', 'carbInput'], inplace=True)

        # Preprocessing Bolus data
        columns_to_impute_with_mean = ['normal', 'bgInput', 'recommended.net', 'recommended.correction', 'insulinOnBoard']
        columns_to_impute_with_mode = ['insulinCarbRatio', 'insulinSensitivityFactor', 'targetBloodGlucose']

        # Imputation
        bolus_data[columns_to_impute_with_mean] = bolus_data[columns_to_impute_with_mean].fillna(bolus_data[columns_to_impute_with_mean].mean())
        bolus_data[columns_to_impute_with_mode] = bolus_data[columns_to_impute_with_mode].fillna(bolus_data[columns_to_impute_with_mode].mode().iloc[0])

        # Merge CGM and Bolus data on 'date'
        merged_data = pd.merge_asof(cgm_data.sort_values('date'), bolus_data.sort_values('date'), on='date', direction='nearest')

        # Create lagged features for forecasting
        lagged_data = create_lag_features(merged_data, 'mg/dl', lags=[1, 2, 3, 4, 5])
        lagged_data.dropna(inplace=True)
        subject_id = os.path.basename(subject_file).replace("Subject", "").replace(".xlsx", "")
        lagged_data['subjectID'] = subject_id

        if lagged_data.empty:
            print(f"No lagged data generated for {subject_file}")
            continue

        # Append to the all_data list
        all_data.append(lagged_data.head(min(Conf.CGM_Inputs,len(lagged_data))))

    except Exception as ex:
        print(f"Error Occurred at {os.path.basename(subject_file)}: {str(ex)}")

# Check if all_data is empty before concatenation
if not all_data:
    print("No data was successfully processed. Exiting.")
else:
    # Concatenate all subjects' data into a single DataFrame
    final_data = pd.concat(all_data, axis=0).reset_index(drop=True)

    # One-hot encoding for categorical columns
    categorical_columns = ['subjectID']
    encoder = OneHotEncoder(drop='first', sparse=False)
    encoded_categorical_data = pd.DataFrame(encoder.fit_transform(final_data[categorical_columns]),
                                            columns=encoder.get_feature_names_out(categorical_columns))

    # Drop original categorical columns and concatenate one-hot encoded columns
    final_data = final_data.drop(columns=categorical_columns)
    final_data = pd.concat([final_data, encoded_categorical_data], axis=1)

    # Define features and target
    X_prophet = final_data['date']
    X = final_data.drop(columns=['date', 'mg/dl'])
    y = final_data['mg/dl']

    print("Dataset shape:", X.shape)

  0%|          | 0/2 [00:00<?, ?it/s]

Processing /content/drive/MyDrive/Upwork clients/ACCOUNT-CREATOR/DiaTrend Data/DiaTrend Data/Sensor Data/Subject1.xlsx...


 50%|█████     | 1/2 [01:25<01:25, 85.41s/it]

Processing /content/drive/MyDrive/Upwork clients/ACCOUNT-CREATOR/DiaTrend Data/DiaTrend Data/Sensor Data/Subject10.xlsx...


100%|██████████| 2/2 [01:48<00:00, 54.31s/it]

Dataset shape: (1000, 14)



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
X.tail()

,normal,insulinCarbRatio,bgInput,recommended.net,recommended.correction,insulinSensitivityFactor,targetBloodGlucose,insulinOnBoard,mg/dl_lag_1,mg/dl_lag_2,mg/dl_lag_3,mg/dl_lag_4,mg/dl_lag_5,subjectID_10
995,11.0,5.5,247.298013,0.0,2.525214,60.0,120.0,0.0,260.0,273.0,282.0,283.0,290.0,1.0
996,11.0,5.5,247.298013,0.0,2.525214,60.0,120.0,0.0,240.0,260.0,273.0,282.0,283.0,1.0
997,11.0,5.5,247.298013,0.0,2.525214,60.0,120.0,0.0,219.0,240.0,260.0,273.0,282.0,1.0
998,11.0,5.5,247.298013,0.0,2.525214,60.0,120.0,0.0,212.0,219.0,240.0,260.0,273.0,1.0
999,11.0,5.5,247.298013,0.0,2.525214,60.0,120.0,0.0,200.0,212.0,219.0,240.0,260.0,1.0


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   normal                    1000 non-null   float64
 1   insulinCarbRatio          1000 non-null   float64
 2   bgInput                   1000 non-null   float64
 3   recommended.net           1000 non-null   float64
 4   recommended.correction    1000 non-null   float64
 5   insulinSensitivityFactor  1000 non-null   float64
 6   targetBloodGlucose        1000 non-null   float64
 7   insulinOnBoard            1000 non-null   float64
 8   mg/dl_lag_1               1000 non-null   float64
 9   mg/dl_lag_2               1000 non-null   float64
 10  mg/dl_lag_3               1000 non-null   float64
 11  mg/dl_lag_4               1000 non-null   float64
 12  mg/dl_lag_5               1000 non-null   float64
 13  subjectID_10              1000 non-null   float64
dtypes: float6

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)

(800, 14) (800,)


In [ ]:
# Create an initial DataFrame to store the results
ResultDF = pd.DataFrame(columns=["modelName", "subjects", "datasetLength", "MAE", "RMSE", "MAPE"])

### Hyperparameter tuning X Model Training

#### XGBoost

In [ ]:
# Define hyperparameter space for XGBoost
space_xgboost = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

def map_params(position):
    indices = [
        min(int(position[0] * len(space_xgboost['n_estimators'])), len(space_xgboost['n_estimators']) - 1),
        min(int(position[1] * len(space_xgboost['max_depth'])), len(space_xgboost['max_depth']) - 1),
        min(int(position[2] * len(space_xgboost['learning_rate'])), len(space_xgboost['learning_rate']) - 1),
        min(int(position[3] * len(space_xgboost['subsample'])), len(space_xgboost['subsample']) - 1),
        min(int(position[4] * len(space_xgboost['colsample_bytree'])), len(space_xgboost['colsample_bytree']) - 1)
    ]
    params = {
        'n_estimators': space_xgboost['n_estimators'][indices[0]],
        'max_depth': space_xgboost['max_depth'][indices[1]],
        'learning_rate': space_xgboost['learning_rate'][indices[2]],
        'subsample': space_xgboost['subsample'][indices[3]],
        'colsample_bytree': space_xgboost['colsample_bytree'][indices[4]]
    }
    return params

def xgb_target_function(position):
    params = map_params(position)
    model = xgb.XGBRegressor(
        verbosity=0,
        objective='reg:squarederror',
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        random_state=42
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    return mse


###################################################

# Configuration for the GWO
dim = 5 # do not change this dimension
n_agents = Conf.number_wolves
n_iter = Conf.iterator
min_values = [0] * dim  # np.zeros(dim)
max_values = [1] * dim  # np.ones(dim)

# Run Grey Wolf Optimizer
best_params, best_score = grey_wolf_optimizer(
    pack_size=n_agents,
    min_values=min_values,
    max_values=max_values,
    iterations=n_iter,
    target_function=xgb_target_function,
    verbose=True
)

print("Best Parameters:", best_params)
print("Best Score (MSE):", best_score)


###################################################

# Train XGBoost with best_params
model = xgb.XGBRegressor(
    verbosity=0,
    objective='reg:squarederror',
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    random_state=42
)

# Fit the model with training data
model.fit(X_train, y_train)

# Predict on the test set
preds = model.predict(X_test)

# Calculate evaluation metrics
mse = mean_squared_error(y_test, preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, preds)
mape = mean_absolute_percentage_error(y_test, preds)

print("MAE:", mae)
print("RMSE:", rmse)
print("MAPE:", mape)

# Append result in dataframe
row_result = ["XGBoost", Conf.Subjects, Conf.Subjects * Conf.CGM_Inputs, mae, rmse, mape]

ResultDF.loc[len(ResultDF)] = row_result

del best_params, best_score
gc.collect()

Iteration 0/3: Best Fitness = 1087.5432631012327
Iteration 1/3: Best Fitness = 59.63463684372655
Iteration 2/3: Best Fitness = 56.408846781468675
Best Parameters: {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.1, 'subsample': 0.6, 'colsample_bytree': 1.0}
Best Score (MSE): 56.408846781468675
MAE: 5.353049716949463
RMSE: 7.510582319731851
MAPE: 0.03776313134872288


3278

#### Random Forest

In [ ]:
# Define the hyperparameter space for Random Forest
space_rf = {
    'n_estimators': [100, 200, 300],  # Number of trees
    'max_depth': [None, 10, 20],  # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4]  # Minimum samples required at a leaf node
}

def map_params(position):
    indices = [
        min(int(position[0] * len(space_rf['n_estimators'])), len(space_rf['n_estimators']) - 1),
        min(int(position[1] * len(space_rf['max_depth'])), len(space_rf['max_depth']) - 1),
        min(int(position[2] * len(space_rf['min_samples_split'])), len(space_rf['min_samples_split']) - 1),
        min(int(position[3] * len(space_rf['min_samples_leaf'])), len(space_rf['min_samples_leaf']) - 1)
    ]
    params = {
        'n_estimators': space_rf['n_estimators'][indices[0]],
        'max_depth': space_rf['max_depth'][indices[1]],
        'min_samples_split': space_rf['min_samples_split'][indices[2]],
        'min_samples_leaf': space_rf['min_samples_leaf'][indices[3]]
    }
    return params
def rf_target_function(position):
    params = map_params(position)
    model = RandomForestRegressor(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        min_samples_split=params['min_samples_split'],
        min_samples_leaf=params['min_samples_leaf'],
        random_state=42
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    return mse


###################################################

# Configuration for the GWO
dim = 4  # Number of hyperparameters for Random Forest
n_agents = Conf.number_wolves
n_iter = Conf.iterator
min_values = [0] * dim  # Lower bounds
max_values = [1] * dim  # Upper bounds

# Run Grey Wolf Optimizer for Random Forest
best_params, best_score = grey_wolf_optimizer(
    pack_size=n_agents,
    min_values=min_values,
    max_values=max_values,
    iterations=n_iter,
    target_function=rf_target_function,
    verbose=True
)

print("Best Parameters:", best_params)
print("Best Score (MSE):", best_score)


###################################################

# Train Random Forest with best_params
model = RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42
)

# Fit the model with training data
model.fit(X_train, y_train)

# Predict on the test set
preds = model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, preds)
mse = mean_squared_error(y_test, preds)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, preds)

print("MAE:", mae)
print("RMSE:", rmse)
print("MAPE:", mape)

# Append result in dataframe
row_result = ["Random Forest", Conf.Subjects, Conf.Subjects * Conf.CGM_Inputs, mae, rmse, mape]
ResultDF.loc[len(ResultDF)] = row_result

del best_params, best_score
gc.collect()

Iteration 0/3: Best Fitness = 49.429258
Iteration 1/3: Best Fitness = 46.6626939041764
Iteration 2/3: Best Fitness = 46.52679304763867
Best Parameters: {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2}
Best Score (MSE): 46.52679304763867
MAE: 4.974294205273293
RMSE: 6.8210551271514195
MAPE: 0.03553166868148823


440

#### LSTM

In [ ]:
# Convert DataFrames to NumPy arrays
X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()

# Print shapes
print("Original X_train_np shape:", X_train_np.shape)
print("Original X_test_np shape:", X_test_np.shape)


###################################################

# Define hyperparameter space for LSTM
space_lstm = {
    'n_units': [50, 100, 150],  # Number of LSTM units
    'learning_rate': [0.001, 0.01, 0.1],  # Learning rates
    'batch_size': [16, 32, 64],  # Batch sizes
    'epochs': [50, 100]  # Number of epochs
}

# Map GWO positions to LSTM hyperparameters correctly for LSTM (4 hyperparameters)
def map_params(position):
    indices = [
        min(int(position[0] * len(space_lstm['n_units'])), len(space_lstm['n_units']) - 1),
        min(int(position[1] * len(space_lstm['learning_rate'])), len(space_lstm['learning_rate']) - 1),
        min(int(position[2] * len(space_lstm['batch_size'])), len(space_lstm['batch_size']) - 1),
        min(int(position[3] * len(space_lstm['epochs'])), len(space_lstm['epochs']) - 1)
    ]
    params = {
        'n_units': space_lstm['n_units'][indices[0]],
        'learning_rate': space_lstm['learning_rate'][indices[1]],
        'batch_size': space_lstm['batch_size'][indices[2]],
        'epochs': space_lstm['epochs'][indices[3]]
    }
    return params

def lstm_target_function(position):
    params = map_params(position)

    # Reshape data for LSTM
    timesteps = 1
    num_samples, num_features = X_train_np.shape
    if num_features % timesteps != 0:
        raise ValueError(f"Number of features ({num_features}) must be divisible by timesteps ({timesteps})")
    n_features = num_features // timesteps

    X_train_reshaped = X_train_np.reshape((X_train_np.shape[0], timesteps, n_features))
    X_test_reshaped = X_test_np.reshape((X_test_np.shape[0], timesteps, n_features))

    # Build LSTM model
    model = Sequential()
    model.add(Input(shape=(timesteps, n_features)))  # Define input shape
    model.add(LSTM(params['n_units'], activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=params['learning_rate']), loss='mse')

    # Train LSTM model
    history = model.fit(X_train_reshaped, y_train, epochs=params['epochs'], batch_size=params['batch_size'], verbose=0, validation_split=0.1)

    # # Inspect training history
    # print("Training Loss History:", history.history['loss'])

    # Predict on the test set
    preds = model.predict(X_test_reshaped).flatten()

    # Replace NaNs in predictions if any
    preds = np.nan_to_num(preds, nan=0.0)

    # Check for NaNs in y_test and preds
    if np.any(np.isnan(y_test)):
        raise ValueError("y_test contains NaN values.")
    if np.any(np.isnan(preds)):
        raise ValueError("Predictions contain NaN values.")

    # Calculate MSE
    mse = mean_squared_error(y_test, preds)
    return mse


###################################################

# Configuration for the GWO
dim = 4  # Number of hyperparameters
n_agents = Conf.number_wolves
n_iter = Conf.iterator_deeplearning
timesteps = (X_train_np.shape[1])
min_values = [0] * dim  # Lower bounds for GWO
max_values = [1] * dim  # Upper bounds for GWO

# Run Grey Wolf Optimizer
best_params, best_score = grey_wolf_optimizer(
    pack_size=n_agents,
    min_values=min_values,
    max_values=max_values,
    iterations=n_iter,
    target_function=lstm_target_function,
    verbose=True
)

print("Best Parameters:", best_params)
print("Best Score (MSE):", best_score)


###################################################

# Use the best parameters found by GWO directly
final_params_lstm = best_params
timesteps = 1
num_samples, n_features = X_train_np.shape

# Reshape input data for final training
X_train_reshaped = X_train_np.reshape((X_train_np.shape[0], timesteps, n_features))
X_test_reshaped = X_test_np.reshape((X_test_np.shape[0], timesteps, n_features))

# Define and compile the LSTM model with the best hyperparameters
model = Sequential()
model.add(Input(shape=(timesteps, n_features)))  # Define input shape
model.add(LSTM(final_params_lstm['n_units'], activation='relu'))
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=final_params_lstm['learning_rate']), loss='mse')


###################################################

# Train the model
model.fit(X_train_reshaped, y_train, epochs=final_params_lstm['epochs'], batch_size=final_params_lstm['batch_size'], verbose=1)

# Predict on the test set
preds = model.predict(X_test_reshaped).flatten()

# Calculate evaluation metrics
mse = mean_squared_error(y_test, preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, preds)
mape = mean_absolute_percentage_error(y_test, preds)

print("Final LSTM Model Metrics:")
print("MAE:", mae)
print("RMSE:", rmse)
print("MAPE:", mape)

# Append result to DataFrame
row_result = ["LSTM", Conf.Subjects, Conf.Subjects * Conf.CGM_Inputs, mae, rmse, mape]
ResultDF.loc[len(ResultDF)] = row_result

del best_params, best_score
gc.collect()

Original X_train_np shape: (800, 14)
Original X_test_np shape: (200, 14)
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Iteration 0/2: Best Fitness = 77.330727918361
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
7/7

1043

#### GRU

In [ ]:
# Convert DataFrames to NumPy arrays
X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()

# Print shapes
print("Original X_train_np shape:", X_train_np.shape)
print("Original X_test_np shape:", X_test_np.shape)

# Define hyperparameter space for GRU
space_gru = {
    'n_units': [50, 100, 150],  # Number of GRU units
    'learning_rate': [0.001, 0.01, 0.1],  # Learning rates
    'batch_size': [16, 32, 64],  # Batch sizes
    'epochs': [50, 100]  # Number of epochs
}

# Map GWO positions to GRU hyperparameters
def map_params(position):
    indices = [
        min(int(position[0] * len(space_gru['n_units'])), len(space_gru['n_units']) - 1),
        min(int(position[1] * len(space_gru['learning_rate'])), len(space_gru['learning_rate']) - 1),
        min(int(position[2] * len(space_gru['batch_size'])), len(space_gru['batch_size']) - 1),
        min(int(position[3] * len(space_gru['epochs'])), len(space_gru['epochs']) - 1)
    ]
    params = {
        'n_units': space_gru['n_units'][indices[0]],
        'learning_rate': space_gru['learning_rate'][indices[1]],
        'batch_size': space_gru['batch_size'][indices[2]],
        'epochs': space_gru['epochs'][indices[3]]
    }
    return params

# Define the target function for the GRU model
def gru_target_function(position):
    params = map_params(position)

    # Reshape data for GRU
    timesteps = 1
    num_samples, num_features = X_train_np.shape
    if num_features % timesteps != 0:
        raise ValueError(f"Number of features ({num_features}) must be divisible by timesteps ({timesteps})")
    n_features = num_features // timesteps

    X_train_reshaped = X_train_np.reshape((X_train_np.shape[0], timesteps, n_features))
    X_test_reshaped = X_test_np.reshape((X_test_np.shape[0], timesteps, n_features))

    # Build GRU model
    model = Sequential()
    model.add(Input(shape=(timesteps, n_features)))  # Define input shape
    model.add(GRU(params['n_units'], activation='relu'))  # Use GRU instead of LSTM
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=params['learning_rate']), loss='mse')

    # Train GRU model
    history = model.fit(X_train_reshaped, y_train, epochs=params['epochs'], batch_size=params['batch_size'], verbose=0, validation_split=0.1)

    # Predict on the test set
    preds = model.predict(X_test_reshaped).flatten()

    # Replace NaNs in predictions if any
    preds = np.nan_to_num(preds, nan=0.0)

    # Check for NaNs in y_test and preds
    if np.any(np.isnan(y_test)):
        raise ValueError("y_test contains NaN values.")
    if np.any(np.isnan(preds)):
        raise ValueError("Predictions contain NaN values.")

    # Calculate MSE
    mse = mean_squared_error(y_test, preds)
    return mse


###################################################

dim = 4  # Number of hyperparameters
n_agents = Conf.number_wolves
n_iter = Conf.iterator_deeplearning
timesteps = 1
min_values = [0] * dim  # Lower bounds for GWO
max_values = [1] * dim  # Upper bounds for GWO

# Run Grey Wolf Optimizer
best_params, best_score = grey_wolf_optimizer(
    pack_size=n_agents,
    min_values=min_values,
    max_values=max_values,
    iterations=n_iter,
    target_function=gru_target_function,
    verbose=True
)


# Use the best parameters found by GWO directly
final_params_gru = best_params
timesteps = 1
num_samples, n_features = X_train_np.shape

print("Best Parameters:", final_params_gru)
print("Best Score (MSE):", best_score)


###################################################

# Reshape input data for final training
X_train_reshaped = X_train_np.reshape((X_train_np.shape[0], timesteps, n_features))
X_test_reshaped = X_test_np.reshape((X_test_np.shape[0], timesteps, n_features))

# Define and compile the GRU model with the best hyperparameters
model = Sequential()
model.add(Input(shape=(timesteps, n_features)))  # Define input shape
model.add(GRU(final_params_gru['n_units'], activation='relu'))  # Use GRU instead of LSTM
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=final_params_gru['learning_rate']), loss='mse')

# Train the model
model.fit(X_train_reshaped, y_train, epochs=final_params_gru['epochs'], batch_size=final_params_gru['batch_size'], verbose=1)

# Predict on the test set
preds = model.predict(X_test_reshaped).flatten()

# Calculate evaluation metrics
mse = mean_squared_error(y_test, preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, preds)
mape = mean_absolute_percentage_error(y_test, preds)

print("Final GRU Model Metrics:")
print("MAE:", mae)
print("RMSE:", rmse)
print("MAPE:", mape)

# Append result to DataFrame
row_result = ["GRU", Conf.Subjects, Conf.Subjects * Conf.CGM_Inputs, mae, rmse, mape]
ResultDF.loc[len(ResultDF)] = row_result

del best_params, best_score
gc.collect()

Original X_train_np shape: (800, 14)
Original X_test_np shape: (200, 14)
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
Iteration 0/2: Best Fitness = 72.55569864051678
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
7/7 

#### Prophet

In [ ]:
# Prepare the data for Prophet
prophet_data = pd.DataFrame({
    'ds': pd.to_datetime(final_data['date']),
    'y': final_data['mg/dl']
})

# Define the hyperparameter space for Prophet
space_prophet = {
    'seasonality_mode': ['additive', 'multiplicative'],
    'seasonality_prior_scale': [1, 10, 100],
    'holidays_prior_scale': [1, 10, 100],
    'changepoint_prior_scale': [0.01, 0.1, 1]
}

# Function to map GWO positions to Prophet parameters
def map_params(position):
    # Ensure positions are within bounds [0, 1]
    position = np.clip(position, 0, 1)

    indices = [
        min(int(position[0] * len(space_prophet['seasonality_mode'])), len(space_prophet['seasonality_mode']) - 1),
        min(int(position[1] * len(space_prophet['seasonality_prior_scale'])), len(space_prophet['seasonality_prior_scale']) - 1),
        min(int(position[2] * len(space_prophet['holidays_prior_scale'])), len(space_prophet['holidays_prior_scale']) - 1),
        min(int(position[3] * len(space_prophet['changepoint_prior_scale'])), len(space_prophet['changepoint_prior_scale']) - 1)
    ]

    params = {
        'seasonality_mode': space_prophet['seasonality_mode'][indices[0]],
        'seasonality_prior_scale': space_prophet['seasonality_prior_scale'][indices[1]],
        'holidays_prior_scale': space_prophet['holidays_prior_scale'][indices[2]],
        'changepoint_prior_scale': space_prophet['changepoint_prior_scale'][indices[3]]
    }
    return params

# Prophet target function for GWO
def prophet_target_function(position):
    params = map_params(position)

    # Initialize Prophet with mapped parameters
    model = Prophet(
        seasonality_mode=params['seasonality_mode'],
        seasonality_prior_scale=params['seasonality_prior_scale'],
        holidays_prior_scale=params['holidays_prior_scale'],
        changepoint_prior_scale=params['changepoint_prior_scale']
    )

    try:
        # Fit the model with the data
        model.fit(prophet_data)

        # Make predictions
        future = model.make_future_dataframe(periods=365)  # Define future periods
        forecast = model.predict(future)

        # Align predictions with actual values
        y_pred = forecast.loc[forecast['ds'].isin(prophet_data['ds']), 'yhat'].values
        y_test = prophet_data['y'].values  # Actual target values

        # Ensure lengths match for evaluation
        if len(y_pred) != len(y_test):
            print("Length mismatch between y_test and y_pred.")
            return float('inf')  # Return a large error if lengths do not match

        # Compute RMSE as the evaluation metric
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        return rmse
    except Exception as e:
        print("Error in Prophet training/prediction:", e)
        return float('inf')  # Return a large error if Prophet fails


###################################################

# GWO Configuration
dim = 4  # Dimensions of parameter space
n_agents = Conf.number_wolves
n_iter = Conf.iterator
min_values = [0] * dim  # Lower bounds
max_values = [1] * dim  # Upper bounds

# Run Grey Wolf Optimizer for Prophet
best_position, best_score = grey_wolf_optimizer(
    pack_size=n_agents,
    min_values=min_values,
    max_values=max_values,
    iterations=n_iter,
    target_function=prophet_target_function,
    verbose=True
)

# Convert the best position to hyperparameters
final_params_prophet = best_position

# Debugging: Print to ensure correct parameters
print("Best Parameters for Prophet:", final_params_prophet)
print("Best Score (RMSE) for Prophet:", best_score)

# Train Prophet with the best hyperparameters
best_model = Prophet(
    seasonality_mode=final_params_prophet['seasonality_mode'],
    seasonality_prior_scale=final_params_prophet['seasonality_prior_scale'],
    holidays_prior_scale=final_params_prophet['holidays_prior_scale'],
    changepoint_prior_scale=final_params_prophet['changepoint_prior_scale']
)

# Fit the model with training data
best_model.fit(prophet_data)

# Make predictions with the best model
future = best_model.make_future_dataframe(periods=365)
forecast = best_model.predict(future)
preds = forecast.loc[forecast['ds'].isin(prophet_data['ds']), 'yhat'].values

# Calculate evaluation metrics
mse = mean_squared_error(prophet_data['y'], preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(prophet_data['y'], preds)
mape = mean_absolute_percentage_error(prophet_data['y'], preds)

print("Final Prophet Model Metrics:")
print("MAE:", mae)
print("RMSE:", rmse)
print("MAPE:", mape)

# Append result to DataFrame
row_result = ["Prophet", Conf.Subjects, Conf.Subjects * Conf.CGM_Inputs, mae, rmse, mape]
ResultDF.loc[len(ResultDF)] = row_result

### Save all the results

In [ ]:
ResultDF

In [ ]:
ResultDF.to_csv('GWO_Model_Training.csv', index=False)